In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torchvision import models
import random
import os
import numpy as np
import pandas as pd
from fastai.vision import ImageList, transform
from functools import partial

from hw_grapheme.densenet import Dnet_1ch

from hw_grapheme.dl_utils.radam import Over9000
from hw_grapheme.loss_func import Loss_combine
from hw_grapheme.callbacks import Metric_idx, Metric_tot, MixUpCallback

from hw_grapheme.dl_utils.csvlogger import CSVLogger
from fastai.basic_train import Learner

from torch import nn
from fastai.callbacks import SaveModelCallback



# import fastai
# import warnings

# import matplotlib.pyplot as plt
# import torch.nn.functional as F

# from sklearn.model_selection import KFold
# from fastai.callbacks import SaveModelCallback, TrackerCallback
# from fastai.callback import Callback
# from torch import nn, Tensor


# from hw_grapheme.dl_utils.radam import RAdam, Over9000
# from hw_grapheme.dl_utils.mish_activation import Mish, to_Mish



# warnings.filterwarnings("ignore")

# fastai.__version__

# from functools import partial
# 


In [3]:
sz = 128
bs = 128
nfolds = 4 #keep the same split as the initial dataset
fold = 1
SEED = 2019
TRAIN = '../data/processed_data/train_images/'
LABELS = '../data/train.csv'

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [4]:
df = pd.read_csv(LABELS)
nunique = list(df.nunique())[1:-1]
print(nunique)
df.head()

[168, 11, 7]


,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [5]:
range(fold*len(df)//nfolds,(fold+1)*len(df)//nfolds)

range(50210, 100420)

In [6]:
stats = ([0.0692], [0.2051])
data = (ImageList.from_df(df, path='.', folder=TRAIN, suffix='.png', 
        cols='image_id', convert_mode='L')
        .split_by_idx(range(fold*len(df)//nfolds,(fold+1)*len(df)//nfolds))
        .label_from_df(cols=['grapheme_root','vowel_diacritic','consonant_diacritic'])
        .transform(transform.get_transforms(do_flip=False,max_warp=0.1), size=sz, padding_mode='zeros')
        .databunch(bs=bs)).normalize(stats)

# data.show_batch()

In [7]:
data

ImageDataBunch;

Train: LabelList (150630 items)
x: ImageList
Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128)
y: MultiCategoryList
,,,,
Path: .;

Valid: LabelList (50210 items)
x: ImageList
Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128)
y: MultiCategoryList
,,,,consonant_diacritic
Path: .;

Test: None

In [8]:
Metric_grapheme = partial(Metric_idx,0)
Metric_vowel = partial(Metric_idx,1)
Metric_consonant = partial(Metric_idx,2)

arch = models.densenet121
model = Dnet_1ch(arch, nunique)

learn = Learner(data, model, loss_func=Loss_combine(), opt_func=Over9000,
        metrics=[Metric_grapheme(),Metric_vowel(),Metric_consonant(),Metric_tot()])

learn.model = nn.DataParallel(learn.model) 


logger = CSVLogger(learn,f'log{fold}')
learn.clip_grad = 1.0
learn.split([model.head1])
learn.unfreeze()

In [9]:
learn.summary()

DataParallel
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [64, 64, 64]         3,136      True      
______________________________________________________________________
BatchNorm2d          [64, 64, 64]         128        True      
______________________________________________________________________
ReLU                 [64, 64, 64]         0          False     
______________________________________________________________________
MaxPool2d            [64, 32, 32]         0          False     
______________________________________________________________________
BatchNorm2d          [64, 32, 32]         128        True      
______________________________________________________________________
ReLU                 [64, 32, 32]         0          False     
______________________________________________________________________
Conv2d               [128, 32, 32]        8,192      True      
_________________________________________________

In [10]:
learn.fit_one_cycle(100, max_lr=slice(0.2e-2,1e-2), wd=[1e-3,0.1e-1], pct_start=0.0, 
    div_factor=100, callbacks = [logger, SaveModelCallback(learn,monitor='metric_tot',
    mode='max',name=f'model_{fold}'),MixUpCallback(learn)])
#metrics: Metric_grapheme, Metric_vowel, Metric_consonant, Metric_tot (competition metric)

epoch,train_loss,valid_loss,metric_idx,metric_idx,metric_idx,metric_tot,time
0,1.290901,0.286785,0.868676,0.950197,0.914374,0.900480,08:59
1,1.166706,0.230460,0.906199,0.964430,0.951510,0.932085,08:24
2,1.106376,0.190903,0.917864,0.968537,0.957484,0.940437,07:51
3,1.042187,0.179430,0.920663,0.973174,0.966635,0.945284,08:04
4,1.007724,0.167739,0.929484,0.970518,0.964455,0.948485,07:56
5,0.961423,0.159344,0.935191,0.975583,0.964749,0.952678,07:55
6,0.924014,0.151804,0.936920,0.977737,0.972259,0.955959,08:02
7,0.883534,0.144951,0.944667,0.977542,0.968594,0.958868,07:45
8,0.854755,0.142849,0.941303,0.977364,0.973060,0.958257,07:54
9,0.822190,0.144102,0.944037,0.975104,0.971543,0.958680,07:44


Better model found at epoch 0 with metric_tot value: 0.9004804491996765.
Better model found at epoch 1 with metric_tot value: 0.9320846796035767.
Better model found at epoch 2 with metric_tot value: 0.9404373168945312.
Better model found at epoch 3 with metric_tot value: 0.945283830165863.
Better model found at epoch 4 with metric_tot value: 0.9484853148460388.
Better model found at epoch 5 with metric_tot value: 0.9526783227920532.
Better model found at epoch 6 with metric_tot value: 0.955958902835846.
Better model found at epoch 7 with metric_tot value: 0.9588677287101746.
Better model found at epoch 10 with metric_tot value: 0.960361659526825.
Better model found at epoch 11 with metric_tot value: 0.9624176025390625.
Better model found at epoch 13 with metric_tot value: 0.9632982015609741.
Better model found at epoch 15 with metric_tot value: 0.9638805985450745.
Better model found at epoch 22 with metric_tot value: 0.9648624062538147.
Better model found at epoch 24 with metric_tot va